In [ ]:
%pip install easyocr

In [10]:
import easyocr

reader = easyocr.Reader(['hi','en'])  # Hindi + English
result = reader.readtext('document.jpeg')

text = " ".join([r[1] for r in result])
print(text)

उसकी माँ और उसका भाई उत्साहित थे| रति और उसकी माँ एक साथ गए| उनके बाद उसके पिताजी और उसका भाई गए| उन्हें एक से बांध दिया गया और उन्हें हवा में फैका गया| वे हवा में उड़ने लगे| वहाँ से उन्हें बहुत सारी डॉल्फिनें दिखीं| वे बहुत उत्साहित हुए व उनका डर खत्म हो गया| रति व उसके परिवार को बहुत आनंद मिला| वे चार दिन बाद बैंगलोर वापस लौटे| पैराशूट


In [ ]:
%pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
        dtype=torch.float16,
            device_map="auto"
            )

In [ ]:
prompt = f"""
You are an OCR correction engine.

- Correct OCR mistakes
- Preserve Hindi/Gujarati language
- Do not translate
- Return only corrected text.

Text:
{text}
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,   # smaller = faster
        do_sample=False,      # deterministic = faster
        temperature=0.0,
        use_cache=True
    )

# Decode ONLY new tokens (IMPORTANT)
generated_tokens = output[0][inputs.input_ids.shape[-1]:]

text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(text)


In [11]:
%pip install edge-tts

Note: you may need to restart the kernel to use updated packages.


In [14]:
import asyncio
import edge_tts

async def generate_audio(text):
    communicate = edge_tts.Communicate(
        text,
        voice="hi-IN-SwaraNeural"
    )
    await communicate.save("./output.mp3")

await generate_audio(text)
